# Section 2.3: Hybrid Quantum-Classical Machine Learning

## Building quantum machine learning models with parameterized circuits

In this section, you'll learn how to combine quantum and classical computing for machine learning tasks. We'll explore how to encode classical data into quantum states, design parameterized quantum circuits (PQCs) for feature extraction, and integrate quantum layers with classical neural networks for binary classification.

## Learning Objectives

By the end of this section, you will be able to:

- Understand the architecture of hybrid quantum-classical machine learning models
- Encode classical data into quantum states using angle and amplitude encoding
- Design parameterized quantum circuits (PQCs) for quantum feature extraction
- Build variational circuits with trainable parameters for machine learning
- Compute expectation values as classical outputs from quantum layers
- Combine quantum feature extraction with classical classification layers
- Understand gradient computation using the parameter-shift rule
- Evaluate quantum ML models on binary classification tasks

In [ ]:
import cirq
import sympy
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple

## 1. Hybrid Quantum-Classical ML Overview

Hybrid quantum-classical machine learning combines quantum circuits for feature extraction with classical neural networks for decision making. This approach leverages the exponentially large Hilbert space of quantum systems while remaining practical on near-term quantum hardware.

In [ ]:
print("Hybrid Quantum-Classical ML Architecture:")
print("")
print("  1. Classical Input Data")
print("       ↓")
print("  2. Data Encoding Layer (Quantum)")
print("       - Maps classical features to quantum states")
print("       ↓")
print("  3. Parameterized Quantum Circuit (PQC)")
print("       - Variational circuit with trainable parameters")
print("       - Extracts quantum features")
print("       ↓")
print("  4. Measurement Layer")
print("       - Converts quantum states to classical expectation values")
print("       ↓")
print("  5. Classical Neural Network")
print("       - Processes quantum features")
print("       ↓")
print("  6. Classification Output")
print("")
print("Training Process:")
print("  - Forward pass through quantum + classical layers")
print("  - Compute loss (e.g., binary cross-entropy)")
print("  - Backpropagate gradients")
print("  - Update both quantum and classical parameters")
print("")
print("Potential Advantages:")
print("  - Access to exponentially large feature spaces")
print("  - Quantum kernels hard to compute classically")
print("  - Native quantum operations may be more efficient")

## 2. Data Encoding Strategies

Data encoding maps classical features into quantum states. The choice of encoding affects both the expressiveness and the hardware requirements of the quantum model.

### 2.1 Angle Encoding

Angle encoding maps each classical feature to a rotation angle on a qubit. This is simple, hardware-efficient, and commonly used in quantum ML. For n features, we need n qubits.

In [ ]:
def angle_encode_data(qubits: List[cirq.Qid], features: List[float]) -> cirq.Circuit:
    """
    Encode classical data as rotation angles.
    
    Each feature becomes a Y-rotation: RY(θ) = cos(θ/2)|0⟩ + sin(θ/2)|1⟩
    """
    if len(qubits) < len(features):
        raise ValueError(f"Need at least {len(features)} qubits for {len(features)} features")
    
    circuit = cirq.Circuit()
    for qubit, feature in zip(qubits, features):
        circuit.append(cirq.ry(feature)(qubit))
    
    return circuit

print("Angle Encoding:")
print("  - Maps each feature to a rotation angle")
print("  - Uses RY(θ) gates: prepares cos(θ/2)|0⟩ + sin(θ/2)|1⟩")
print("  - Simple and hardware-efficient")
print("  - Circuit depth: O(n) for n features")
print("")

# Demonstrate angle encoding
qubits = cirq.LineQubit.range(3)
test_features = [np.pi/4, np.pi/3, np.pi/2]
encoding_circuit = angle_encode_data(qubits, test_features)

print(f"Example - Encoding features {test_features}:")
print(encoding_circuit)
print("")

# Show the quantum state prepared
simulator = cirq.Simulator()
result = simulator.simulate(encoding_circuit)
print("Resulting quantum state (first 4 amplitudes):")
for i in range(min(4, len(result.final_state_vector))):
    basis_state = format(i, f'0{len(qubits)}b')
    amplitude = result.final_state_vector[i]
    prob = abs(amplitude) ** 2
    print(f"  |{basis_state}⟩: amplitude = {amplitude.real:.4f}, probability = {prob:.4f}")

### 2.2 Amplitude Encoding

Amplitude encoding maps features directly to quantum state amplitudes. For n qubits, we can encode 2^n values exponentially compactly, but this requires complex state preparation circuits.

In [ ]:
def amplitude_encode_data(qubits: List[cirq.Qid], data: np.ndarray) -> cirq.Circuit:
    """
    Encode classical data into quantum state amplitudes.
    
    For n qubits, can encode 2^n values. Data must be normalized.
    This simple implementation works for 2 amplitudes (1 qubit).
    """
    # Verify normalization
    norm = np.linalg.norm(data)
    if not np.isclose(norm, 1.0):
        raise ValueError(f"Data must be normalized (norm={norm:.4f})")
    
    circuit = cirq.Circuit()
    
    # Simple case: encode 2D data into 1 qubit
    # data = [a, b] -> |ψ⟩ = a|0⟩ + b|1⟩
    if len(qubits) == 1 and len(data) == 2:
        if np.abs(data[0]) > 1e-10:
            theta = 2 * np.arctan2(data[1], data[0])
            circuit.append(cirq.ry(theta)(qubits[0]))
    else:
        raise NotImplementedError(
            "Amplitude encoding for >2 amplitudes requires advanced state preparation."
        )
    
    return circuit

print("Amplitude Encoding:")
print("  - Maps features to quantum state amplitudes")
print("  - Exponentially compact: 2^n amplitudes for n qubits")
print("  - Requires complex state preparation circuits")
print("  - For n qubits: |ψ⟩ = Σᵢ aᵢ|i⟩ where aᵢ are features")
print("")

# Demonstrate amplitude encoding (simple 2D case)
q = cirq.LineQubit(0)
data_2d = np.array([0.6, 0.8])  # Already normalized: 0.6² + 0.8² = 1
amp_circuit = amplitude_encode_data([q], data_2d)

print(f"Example - Encoding normalized data {data_2d}:")
print(amp_circuit)
print("")

# Verify the state
result = simulator.simulate(amp_circuit)
state_vector = result.final_state_vector
print("Resulting state:")
print(f"  |0⟩: {state_vector[0].real:.4f} (target: {data_2d[0]:.4f})")
print(f"  |1⟩: {state_vector[1].real:.4f} (target: {data_2d[1]:.4f})")
print(f"  Encoding successful: {np.allclose(np.abs(state_vector), data_2d)}")

## 3. Parameterized Quantum Circuits (PQCs)

A PQC contains trainable parameters that are optimized during training. It acts as the quantum feature extractor in the hybrid model. The circuit alternates between parameterized rotations and fixed entangling gates.

In [ ]:
def build_pqc(qubits: List[cirq.Qid], num_layers: int = 1) -> Tuple[cirq.Circuit, List[sympy.Symbol]]:
    """
    Build a parameterized quantum circuit for feature extraction.
    
    Architecture:
      1. Parameterized single-qubit rotations (trainable)
      2. Entangling two-qubit gates (fixed)
      3. Repeat for num_layers
    
    This is a hardware-efficient ansatz suitable for NISQ devices.
    """
    circuit = cirq.Circuit()
    params = []
    
    n_qubits = len(qubits)
    param_idx = 0
    
    for layer in range(num_layers):
        # Layer of parameterized rotations
        for i, qubit in enumerate(qubits):
            # Each qubit gets three rotation parameters (full single-qubit control)
            theta = sympy.Symbol(f'theta_{param_idx}')
            phi = sympy.Symbol(f'phi_{param_idx + 1}')
            lam = sympy.Symbol(f'lambda_{param_idx + 2}')
            
            params.extend([theta, phi, lam])
            param_idx += 3
            
            # Apply rotations: RZ(θ)RY(φ)RZ(λ)
            circuit.append([
                cirq.rz(theta)(qubit),
                cirq.ry(phi)(qubit),
                cirq.rz(lam)(qubit)
            ])
        
        # Layer of entangling gates (ring topology)
        for i in range(n_qubits):
            control = qubits[i]
            target = qubits[(i + 1) % n_qubits]
            circuit.append(cirq.CZ(control, target))
    
    return circuit, params

print("Parameterized Quantum Circuit (PQC) Structure:")
print("")

qubits = cirq.LineQubit.range(4)
pqc, params = build_pqc(qubits, num_layers=2)

print(f"Configuration:")
print(f"  - Number of qubits: {len(qubits)}")
print(f"  - Number of layers: 2")
print(f"  - Total parameters: {len(params)}")
print(f"  - Circuit depth: {len(pqc)}")
print("")

print("Each layer contains:")
print("  1. Parameterized rotations: RZ(θ)RY(φ)RZ(λ) on each qubit")
print("     - Provides full single-qubit control")
print("     - 3 parameters per qubit per layer")
print("  2. Entangling gates: CZ between adjacent qubits (ring topology)")
print("     - Creates quantum correlations")
print("     - Enables non-linear feature maps")
print("")

print("Circuit diagram (first 15 moments):")
print(pqc[:15])
print("...")
print("")

print(f"Symbolic parameters (first 6):")
for i, param in enumerate(params[:6]):
    print(f"  {param}")

## 4. Measurement and Expectation Values

To extract classical information from quantum states, we measure observables and compute expectation values. For machine learning, expectation values ⟨ψ|O|ψ⟩ serve as the classical outputs from the quantum layer.

In [ ]:
def compute_expectation_values(
    circuit: cirq.Circuit,
    observables: List[cirq.PauliString]
) -> np.ndarray:
    """
    Compute expectation values of observables on circuit output state.
    
    Expectation values are classical numbers extracted from quantum states.
    """
    if cirq.is_parameterized(circuit):
        raise ValueError("Circuit must be resolved before computing expectations")
    
    simulator = cirq.Simulator()
    expectations = []
    
    for observable in observables:
        # Compute ⟨ψ|O|ψ⟩
        result = simulator.simulate_expectation_values(
            circuit,
            observables=[observable]
        )
        # Hermitian observables have real expectation values
        expectations.append(np.real(result[0]))
    
    return np.array(expectations)

print("Expectation Values as Quantum Features:")
print("")
print("For an observable O and state |ψ⟩:")
print("  ⟨O⟩ = ⟨ψ|O|ψ⟩")
print("")
print("Properties:")
print("  - Real-valued for Hermitian observables (e.g., Pauli operators)")
print("  - Bounded: for single-qubit Z, ⟨Z⟩ ∈ [-1, 1]")
print("  - Differentiable w.r.t. circuit parameters")
print("")

# Demonstrate expectation value computation
print("Example - Computing Z expectation values:")
print("")

qubits = cirq.LineQubit.range(3)
# Create a simple circuit
circuit = cirq.Circuit(
    cirq.H(qubits[0]),           # |+⟩ state
    cirq.X(qubits[1]),           # |1⟩ state
    cirq.ry(np.pi/3)(qubits[2])  # Superposition
)

print("Circuit:")
print(circuit)
print("")

# Define observables: Z on each qubit
observables = [cirq.Z(q) for q in qubits]
expectations = compute_expectation_values(circuit, observables)

print("Z expectation values:")
print(f"  ⟨Z₀⟩ = {expectations[0]:.4f}  (|+⟩ state: expect ~0)")
print(f"  ⟨Z₁⟩ = {expectations[1]:.4f}  (|1⟩ state: expect -1)")
print(f"  ⟨Z₂⟩ = {expectations[2]:.4f}  (rotation state)")
print("")
print("These expectation values become features for the classical layer.")

## 5. Complete Quantum Layer

A quantum layer combines data encoding, parameterized circuit application, and measurement into a single forward pass. This is the quantum component of the hybrid model.

In [ ]:
def quantum_layer(
    qubits: List[cirq.Qid],
    input_data: np.ndarray,
    num_layers: int = 1,
    parameter_values: np.ndarray = None
) -> np.ndarray:
    """
    Execute quantum layer: encode data, apply PQC, measure observables.
    
    This is the forward pass of a quantum layer.
    """
    # Step 1: Encode input data
    encoding_circuit = angle_encode_data(qubits, input_data[:len(qubits)])
    
    # Step 2: Build and resolve PQC
    pqc, params = build_pqc(qubits, num_layers)
    
    if parameter_values is None:
        # Initialize with random parameters
        parameter_values = np.random.uniform(0, 2 * np.pi, size=len(params))
    
    param_dict = {param: val for param, val in zip(params, parameter_values)}
    resolved_pqc = cirq.resolve_parameters(pqc, param_dict)
    
    # Step 3: Combine circuits
    full_circuit = encoding_circuit + resolved_pqc
    
    # Step 4: Define observables (Z on each qubit)
    observables = [cirq.Z(q) for q in qubits]
    
    # Step 5: Compute expectation values
    expectations = compute_expectation_values(full_circuit, observables)
    
    return expectations

print("Quantum Layer Forward Pass:")
print("")
print("  Input: Classical feature vector")
print("    ↓")
print("  Step 1: Angle encoding (map to quantum state)")
print("    ↓")
print("  Step 2: Apply parameterized quantum circuit")
print("    ↓")
print("  Step 3: Measure expectation values")
print("    ↓")
print("  Output: Classical feature vector (quantum features)")
print("")

# Demonstrate quantum layer
np.random.seed(42)
qubits = cirq.LineQubit.range(4)
input_features = np.array([0.5, 1.2, 0.8, 1.5])

print(f"Example - Processing input features:")
print(f"  Input: {input_features}")
print("")

quantum_features = quantum_layer(qubits, input_features, num_layers=2)

print(f"  Quantum features (expectation values): {quantum_features}")
print("")
print(f"Properties of quantum features:")
print(f"  - Dimension: {len(quantum_features)} (one per qubit)")
print(f"  - Range: [{quantum_features.min():.3f}, {quantum_features.max():.3f}]")
print(f"  - Mean: {quantum_features.mean():.3f}")
print("")
print("These quantum features will be fed to the classical layer.")

## 6. Classical Layer for Classification

The classical layer processes quantum features to produce classification outputs. For binary classification, we use a linear layer with sigmoid activation.

In [ ]:
def classify_binary(quantum_output: np.ndarray, weights: np.ndarray = None) -> float:
    """
    Apply classical layer for binary classification.
    
    Takes quantum expectation values and produces classification probability.
    """
    if weights is None:
        # Simple average of quantum outputs
        weights = np.ones(len(quantum_output)) / len(quantum_output)
    
    # Linear combination
    linear_output = np.dot(weights, quantum_output)
    
    # Sigmoid activation: σ(x) = 1/(1 + e^(-x))
    if linear_output >= 0:
        probability = 1 / (1 + np.exp(-linear_output))
    else:
        exp_x = np.exp(linear_output)
        probability = exp_x / (1 + exp_x)
    
    return probability

print("Classical Classification Layer:")
print("")
print("Architecture:")
print("  Quantum features → Linear layer → Sigmoid → Probability")
print("")
print("Mathematical form:")
print("  output = σ(w·f + b)")
print("  where:")
print("    f = quantum features (expectation values)")
print("    w = trainable weights")
print("    b = trainable bias")
print("    σ = sigmoid function")
print("")

# Demonstrate classification
print("Example - Binary classification:")
print("")

quantum_features = np.array([0.5, -0.3, 0.8, 0.1])
weights = np.array([0.7, 0.3, 0.5, 0.2])

print(f"Quantum features: {quantum_features}")
print(f"Weights: {weights}")
print("")

probability = classify_binary(quantum_features, weights)
predicted_class = 1 if probability > 0.5 else 0

print(f"Linear combination: {np.dot(weights, quantum_features):.4f}")
print(f"Classification probability: {probability:.4f}")
print(f"Predicted class: {predicted_class} (threshold = 0.5)")
print("")
print("During training, weights are optimized to minimize classification error.")

## 7. Complete Hybrid Model Demonstration

Let's demonstrate the complete hybrid quantum-classical model on a toy binary classification problem.

In [ ]:
print("=" * 70)
print("HYBRID QUANTUM-CLASSICAL BINARY CLASSIFICATION")
print("=" * 70)
print("")

# Setup
np.random.seed(42)
num_qubits = 4
num_samples = 8
num_layers = 2

print(f"Configuration:")
print(f"  - Qubits: {num_qubits}")
print(f"  - PQC layers: {num_layers}")
print(f"  - Training samples: {num_samples}")
print("")

# Generate toy dataset
# Class 0: small feature values [0, π/2]
# Class 1: large feature values [π/2, π]
X_class0 = np.random.uniform(0, np.pi/2, size=(num_samples//2, num_qubits))
X_class1 = np.random.uniform(np.pi/2, np.pi, size=(num_samples//2, num_qubits))

X = np.vstack([X_class0, X_class1])
y = np.array([0] * (num_samples//2) + [1] * (num_samples//2))

print("Dataset Generated:")
print(f"  Class 0: {len(X_class0)} samples (features ∈ [0, π/2])")
print(f"  Class 1: {len(X_class1)} samples (features ∈ [π/2, π])")
print("")

# Build model components
qubits = cirq.LineQubit.range(num_qubits)
pqc, params = build_pqc(qubits, num_layers)

print("Model Architecture:")
print(f"  1. Input Layer: {num_qubits} classical features")
print(f"  2. Data Encoding: Angle encoding (RY rotations)")
print(f"  3. Quantum Layer: {num_layers} layers of rotations + entanglement")
print(f"  4. Quantum Parameters: {len(params)} trainable angles")
print(f"  5. Measurement: Expectation values of {num_qubits} Z observables")
print(f"  6. Classical Layer: Dense layer with sigmoid activation")
print(f"  7. Output: Binary classification probability")
print("")

print("-" * 70)
print("FORWARD PASS DEMONSTRATION")
print("-" * 70)
print("")

# Process samples through hybrid model
for i in range(min(4, num_samples)):
    print(f"Sample {i+1} (True label: {y[i]}):")
    print(f"  Input features: [{', '.join(f'{x:.3f}' for x in X[i])}]")
    
    # Quantum layer forward pass
    quantum_features = quantum_layer(qubits, X[i], num_layers=num_layers)
    print(f"  Quantum features: [{', '.join(f'{q:.3f}' for q in quantum_features)}]")
    
    # Classical layer
    prediction = classify_binary(quantum_features)
    predicted_class = 1 if prediction > 0.5 else 0
    
    print(f"  Prediction probability: {prediction:.4f}")
    print(f"  Predicted class: {predicted_class}")
    
    # Check correctness
    correct = "✓" if predicted_class == y[i] else "✗"
    print(f"  Result: {correct}")
    print("")

## 8. Training Process and Gradient Computation

Training hybrid quantum-classical models requires computing gradients with respect to both quantum circuit parameters and classical network weights. For quantum circuits, we use the **parameter-shift rule**.

In [ ]:
print("Training Hybrid Quantum-Classical Models:")
print("")
print("Training Loop:")
print("  1. Forward Pass:")
print("     - Encode classical data into quantum states")
print("     - Apply parameterized quantum circuit")
print("     - Measure expectation values (quantum features)")
print("     - Apply classical layers to get predictions")
print("")
print("  2. Compute Loss:")
print("     - Compare predictions to true labels")
print("     - Common loss: binary cross-entropy")
print("     - L = -[y log(p) + (1-y) log(1-p)]")
print("")
print("  3. Backward Pass:")
print("     - Compute gradients of loss w.r.t. all parameters")
print("     - Classical gradients: standard backpropagation")
print("     - Quantum gradients: parameter-shift rule")
print("")
print("  4. Update Parameters:")
print("     - Adjust quantum circuit parameters (rotation angles)")
print("     - Adjust classical layer weights and biases")
print("     - Use optimizer (e.g., Adam, SGD)")
print("")
print("  5. Repeat until convergence")
print("")

print("-" * 70)
print("Parameter-Shift Rule for Quantum Gradients")
print("-" * 70)
print("")
print("For a rotation gate R(θ) and observable O:")
print("")
print("  ∂⟨O⟩/∂θ = [⟨O⟩(θ + π/2) - ⟨O⟩(θ - π/2)] / 2")
print("")
print("Properties:")
print("  - Requires two circuit evaluations per parameter")
print("  - Exact (not approximate like finite differences)")
print("  - Works on real quantum hardware")
print("  - No need to backpropagate through quantum circuit")
print("")

# Demonstrate parameter-shift rule
print("Example - Computing gradient for single rotation:")
print("")

q = cirq.LineQubit(0)
theta_symbol = sympy.Symbol('theta')
circuit_template = cirq.Circuit(cirq.ry(theta_symbol)(q))
observable = cirq.Z(q)

# Choose parameter value
theta_val = np.pi / 4
shift = np.pi / 2

# Evaluate at θ + π/2
circuit_plus = cirq.resolve_parameters(circuit_template, {theta_symbol: theta_val + shift})
result_plus = simulator.simulate_expectation_values(circuit_plus, observables=[observable])
exp_plus = np.real(result_plus[0])

# Evaluate at θ - π/2
circuit_minus = cirq.resolve_parameters(circuit_template, {theta_symbol: theta_val - shift})
result_minus = simulator.simulate_expectation_values(circuit_minus, observables=[observable])
exp_minus = np.real(result_minus[0])

# Compute gradient
gradient = (exp_plus - exp_minus) / 2

print(f"Circuit: RY(θ)|0⟩, Observable: Z")
print(f"Parameter value: θ = {theta_val:.4f}")
print("")
print(f"⟨Z⟩(θ + π/2) = {exp_plus:.4f}")
print(f"⟨Z⟩(θ - π/2) = {exp_minus:.4f}")
print("")
print(f"Gradient: ∂⟨Z⟩/∂θ = {gradient:.4f}")
print("")
print("This gradient is used to update θ during training.")
print("")

print("-" * 70)
print("TensorFlow Quantum Integration")
print("-" * 70)
print("")
print("With TensorFlow Quantum (TFQ), training is automatic:")
print("")
print("  import tensorflow as tf")
print("  import tensorflow_quantum as tfq")
print("")
print("  # Create quantum layer")
print("  quantum_layer = tfq.layers.PQC(pqc, observables)")
print("")
print("  # Build hybrid model")
print("  model = tf.keras.Sequential([")
print("      quantum_layer,")
print("      tf.keras.layers.Dense(1, activation='sigmoid')")
print("  ])")
print("")
print("  # Compile and train")
print("  model.compile(optimizer='adam', loss='binary_crossentropy')")
print("  model.fit(X_train, y_train, epochs=50)")
print("")
print("TFQ automatically:")
print("  - Computes gradients using parameter-shift rule")
print("  - Integrates with TensorFlow's automatic differentiation")
print("  - Enables end-to-end training of hybrid models")
print("  - Supports GPU acceleration and batch processing")

## 9. Quantum Advantage in Machine Learning

Understanding when quantum ML might provide advantages over classical approaches is crucial for choosing the right tool for the job.

In [ ]:
print("=" * 70)
print("QUANTUM ADVANTAGE IN MACHINE LEARNING")
print("=" * 70)
print("")

print("Potential Advantages:")
print("")
print("  1. Feature Space Dimensionality")
print("     - Quantum states live in exponentially large Hilbert space")
print("     - n qubits → 2^n dimensional state space")
print("     - May access feature representations unreachable classically")
print("     - Example: 50 qubits → 10^15 dimensional space")
print("")
print("  2. Quantum Kernel Methods")
print("     - Quantum kernels K(x,x') = |⟨φ(x)|φ(x')⟩|²")
print("     - Some quantum kernels are hard to compute classically")
print("     - May enable quantum advantage for certain learning tasks")
print("     - Active area of research")
print("")
print("  3. Hardware Efficiency")
print("     - Native quantum operations may be more efficient")
print("     - Potential for quantum speedup on specific problems")
print("     - Particularly for quantum data or quantum simulation")
print("")

print("-" * 70)
print("")

print("Current Limitations (NISQ Era):")
print("")
print("  1. Hardware Constraints")
print("     - Limited qubit count (< 1000 qubits currently)")
print("     - Limited circuit depth due to decoherence")
print("     - High gate error rates (~0.1% - 1%)")
print("     - Restricted qubit connectivity")
print("")
print("  2. Noise and Errors")
print("     - Noise degrades quantum advantage")
print("     - Error mitigation adds overhead")
print("     - Training may converge to local minima")
print("")
print("  3. Classical Competition")
print("     - Classical ML is highly optimized")
print("     - Deep learning on GPUs is very efficient")
print("     - Classical simulation often competitive for small problems")
print("     - Theoretical advantage not yet demonstrated practically")
print("")
print("  4. Problem Encoding")
print("     - Encoding classical data into quantum states is non-trivial")
print("     - May require deep circuits (costly on NISQ hardware)")
print("     - Measurement collapse limits information extraction")
print("")

print("-" * 70)
print("")

print("Promising Application Areas:")
print("")
print("  1. Quantum Chemistry and Materials Science")
print("     - Learning molecular properties")
print("     - Drug discovery and design")
print("     - Natural quantum data")
print("")
print("  2. Quantum State Tomography")
print("     - Learning to reconstruct quantum states")
print("     - Quantum process tomography")
print("     - Inherently quantum tasks")
print("")
print("  3. Optimization Problems")
print("     - QAOA for combinatorial optimization")
print("     - Variational algorithms")
print("     - Hybrid quantum-classical optimization")
print("")
print("  4. Generative Models")
print("     - Quantum GANs")
print("     - Quantum Boltzmann machines")
print("     - Learning quantum distributions")
print("")

print("Bottom line: Quantum ML is promising but still experimental.")
print("Focus on problems where quantum properties are essential.")

## Exercises

1. **Custom PQC Design**: Design a PQC with different entangling patterns (e.g., all-to-all connectivity instead of ring). Compare the expressiveness and circuit depth.

2. **Data Encoding Comparison**: Implement a quantum layer using amplitude encoding instead of angle encoding. What are the trade-offs in circuit depth and data capacity?

3. **Multi-class Classification**: Extend the binary classifier to handle 3 classes. How many observables do you need? What activation function should replace sigmoid?

4. **Parameter-Shift Rule**: Implement gradient computation using the parameter-shift rule for a simple circuit. Verify that it matches numerical gradients (finite differences).

5. **Feature Visualization**: Extract quantum features from multiple data points and visualize them using PCA or t-SNE. Do different classes cluster in the quantum feature space?

6. **Circuit Depth Analysis**: How does classification accuracy change as you increase the number of PQC layers? What happens when circuits become too deep for NISQ hardware?

7. **Measurement Strategy**: Instead of measuring Z on all qubits, try measuring different Pauli operators (X, Y, Z) on each qubit. Does this improve classification performance?

8. **Quantum vs Classical**: Train both a quantum model and a classical neural network on the same dataset. Compare accuracy, training time, and model size.

## Key Takeaways

- **Hybrid quantum-classical models** combine quantum circuits for feature extraction with classical neural networks for decision making
- **Data encoding** maps classical features to quantum states; common strategies include angle encoding and amplitude encoding
- **Parameterized quantum circuits (PQCs)** contain trainable rotation angles optimized during training
- **Expectation values** ⟨ψ|O|ψ⟩ extract classical information from quantum states and serve as quantum features
- **Parameter-shift rule** enables gradient computation for quantum circuits without backpropagation
- **TensorFlow Quantum** provides end-to-end integration for training hybrid quantum-classical models
- **Quantum advantage** in ML remains an open question; focus on problems with inherently quantum structure
- **NISQ limitations** (noise, limited qubits, shallow circuits) constrain current quantum ML applications
- Hardware-efficient ansätze with shallow circuits and minimal entangling gates are crucial for near-term devices
- Training hybrid models requires optimizing both quantum circuit parameters and classical network weights simultaneously

---

**Previous:** [Section 2.2 - VQE for Quantum Chemistry](part2_section_2_2_vqe.ipynb)

**Next:** [Section 2.4 - Error Mitigation Techniques](part2_section_2_4_error_mitigation.ipynb) - Learn strategies for mitigating errors on noisy quantum hardware